In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
client = Client(cluster)
client
import dask_cudf as pd
#lol numpy



/home/karl/anaconda3/envs/rapids-21.06/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39891 instead
  expected, actual


In [6]:
#mrconso_columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS']
#mrconso_columns = ['CUI', 'LAT', 'TS', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPRESS', 'CVF']

mrconso_columns = ["CUI","LAT","TS","LUI","STT","SUI","ISPREF","AUI","SAUI","SCUI","SDUI","SAB","TTY","CODE","STR","SRL","SUPRESS","CVF",]
#mrrel_columns = ['CUI1', 'REL', 'CUI2', 'RELA', 'SAB', 'SL', 'MG']
mrrel_columns = ['CUI', 'AUI', 'STYPE', 'REL', 'CUI2', 'AUI2', 'STYPE2', 'RELA', 'RUI', 'SRUI', 'SAB', 'SL','RG','DIR','SUPPRESS','CVF']

In [7]:
mrconso_iter = pd.read_csv(
    "../UMLS/MRCONSO.RRF",
    sep="|",
    names=mrconso_columns,
    dtype=str
)
mrconso = mrconso_iter[mrconso_iter["LAT"] == "ENG"]
# mrconso = mrconso.map_partitions(lambda df: df.to_pandas())

mrrel_iter = pd.read_csv(
    "../UMLS/MRREL.RRF",
    sep="|",
    names=mrrel_columns,
)

#mrrel = mrrel_iter[mrrel_iter["RELA"] == "isa"]
# mrrel = mrrel.map_partitions(lambda df: df.to_pandas())

ValueError: Metadata inference failed in `map`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ValueError('Reindexing only valid with uniquely valued Index objects')

Traceback:
---------
  File "/home/karl/anaconda3/envs/rapids-21.06/lib/python3.7/site-packages/dask/dataframe/utils.py", line 175, in raise_on_meta_error
    yield
  File "/home/karl/anaconda3/envs/rapids-21.06/lib/python3.7/site-packages/dask/dataframe/core.py", line 5546, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/home/karl/anaconda3/envs/rapids-21.06/lib/python3.7/site-packages/dask/utils.py", line 900, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "/home/karl/anaconda3/envs/rapids-21.06/lib/python3.

In [ ]:

mrrel = mrrel_iter
#mrrel = mrrel.drop_duplicates(subset=["CUI1"])
#mrconso = mrconso.drop_duplicates(subset=['CUI'])
mapper = mrconso.set_index('CUI')['STR'].compute()
merged = mrrel.copy()
merged['hyponym'] = merged['CUI'].map(mapper)
merged['hypernym'] = merged['CUI2'].map(mapper)
drop_columns = ['CUI', 'AUI', 'STYPE', 'REL', 'CUI2', 'AUI2', 'STYPE2', 'RELA', 'RUI', 'SRUI', 'SAB', 'SL','RG','DIR','SUPPRESS','CVF']
merged = merged.drop(drop_columns, axis=1)
merged_pandas = merged.to_pandas()
merged_pandas.to_csv('sub_output.csv', sep='|', index=False)